In [19]:
from sympy import symbols, init_printing
import sympy
import sympy.physics.mechanics as me
from pydy.system import System
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.integrate import ode, odeint
from sympy.utilities.lambdify import lambdastr
from scipy.optimize import fsolve
import numpy as np
from CDPMv2_animation import animate as ani
%matplotlib inline
init_printing(True)

In [20]:
workspace_width = 10.4
middle = workspace_width/2

plate_width = 0.5
plate_height = 0.3

rod_length = 2.91
rod_radius = 0.02
e_offset = 1.60675

mass_of_plate = 10
mass_of_rod = 2

cable_K = 1000.0
cable_C = 0.7
rod_k = 1000.0
rod_c = 5.0

In [160]:
init_x = middle - middle/2
init_z = 1.5
init_beta = 0

In [166]:
# Create the frames
A = me.ReferenceFrame('A')
# B = me.ReferenceFrame('B')
# C = me.ReferenceFrame('C')

# Create the symbols
# x, z, e, theta, beta = me.dynamicsymbols('x z e theta beta')
x_dot, z_dot, e_dot = me.dynamicsymbols('x_dot z_dot e_dot')
theta_dot, beta_dot = me.dynamicsymbols('theta_dot beta_dot')
H, a, b, M, m, g, k, t = sympy.symbols('H a b M m g k t')
Ip, Ir, c, r, p, kr, cr, D = sympy.symbols('Ip Ir c r p kr cr D')
L1, L2, X, Z, BETA = sympy.symbols('L1 L2 X Z BETA')

In [167]:
for j in range(2):
    if j == 0:
        x, z, e, theta, beta = sympy.symbols('x z e theta beta')
        # Orient the Beta frame
        B = A.orientnew('B', 'Axis', [beta, A.y])
        C = A.orientnew('C', 'Axis', [theta, A.y])
        B.set_ang_vel(A, beta_dot * A.y)
        C.set_ang_vel(A, theta_dot * A.y)
    else:
        x, z, e, theta, beta = me.dynamicsymbols('x z e theta beta')
        # Orient the Beta frame
        B = A.orientnew('B', 'Axis', [beta, A.y])
        C = A.orientnew('C', 'Axis', [theta, A.y])
        B.set_ang_vel(A, beta_dot * A.y)
        C.set_ang_vel(A, theta_dot * A.y)
    # Create the origin points point
    A1 = me.Point('A1')
    A2 = me.Point('A2')

    # Set the origin points positions
    A1.set_pos(A1, 0)
    A2.set_pos(A1, H * A.x)

    # Create the plate center of Gravity
    G = me.Point('G')
    Gr = me.Point('Gr')

    # Set the COG position
    G.set_pos(A1, x*A.x + z*A.z)
    Gr.set_pos(G, e * C.z)

    # Create the attachment points
    B1 = me.Point('B1')
    B2 = me.Point('B2')

    # # Set the attachment points positions
    B1.set_pos(G, -a/2 * B.x - b/2 * B.z)
    B2.set_pos(G, a/2 * B.x - b/2 * B.z)

    # Create Rod top and Bottom points
    C1 = me.Point('C1')
    C2 = me.Point('C2')
    C1.set_pos(Gr, -D/2 * C.z)
    C2.set_pos(Gr, D/2 * C.z)

    # Create the position vectors
    a2 = A2.pos_from(A1)
    a2_x = a2 & A.x
    a2_z = a2 & A.z

    r1 = B1.pos_from(G)
    r1_x = r1 & A.x
    r1_z = r1 & A.z

    r2 = B2.pos_from(G)
    r2_x = r2 & A.x
    r2_z = r2 & A.z

    r21 = B2.pos_from(B1)
    r21_x = r21 & A.x
    r21_z = r21 & A.z

    s1 = B1.pos_from(A1)
    s2 = B2.pos_from(A2)

    # Calculating the Geometric lengths from the top corners of the plate
    Length1 = s1.magnitude()
    Length2 = s2.magnitude()

    # Creating the unit vectors pointing from the origins to the top plate points
    s1_vector = s1.normalize()
    s2_vector = s2.normalize()
    
    spF1 = A1.pos_from(B1)
    spF2 = A2.pos_from(B2)
    
    if j == 0:
        p = ((r21_x * x + a2_x * r1_x) * z - r21_z * x**2 +
             (r1_z * r2_x - r1_x * r2_z + a2_x * r21_z - a2_z * r2_x) * x +
             r1_x *  (a2_x * r2_z - a2_z * r2_x))
        equation_all = p.subs({H:workspace_width, a:plate_width, b:plate_height}).evalf()
        equation_all = equation_all.simplify()
        lambda_str1 = lambdastr((beta), equation_all)
        lambda_str2 = lambda_str1.replace('sin', 'np.sin')
        lambda_str3 = lambda_str2.replace('cos', 'np.cos')
        lambda_str4 = lambda_str3.replace('x', 'x_temp')
        lambda_str = lambda_str4.replace('z', 'z_temp')
        func1 = eval(lambda_str)

In [168]:
# Getting the geometric lengths for testing
geo_Len1_init = Length1.subs({x:init_x,
                     z:init_z,
                     H:workspace_width,
                     a:plate_width,
                     b:plate_height, 
                     beta:init_beta})
geo_Len2_init = Length2.subs({x:init_x,
                     z:init_z,
                     H:workspace_width,
                     a:plate_width,
                     b:plate_height, 
                     beta:init_beta})

e_init = ((9.81 * mass_of_rod) / rod_k) - e_offset
print('These are the geometric lengths\n----------------\
        \nInitial Length 1: {}\nInitial Length 2: {}\ne:{}'.format(geo_Len1_init,
                                                                   geo_Len2_init,
                                                                   e_init))

These are the geometric lengths
----------------        
Initial Length 1: 2.71016604657353
Initial Length 2: 7.66974575849813
e:-1.58713


In [169]:
fake_length1, fake_length2 = sympy.symbols('fake_length1 fake_length2')

Fsp1 = 1000*(geo_Len1_init - fake_length1)*spF1
Fsp2 = 1000*(geo_Len2_init - fake_length2)*spF2

Fsp1_x = Fsp1 & A.x
Fsp1_z = Fsp1 & A.z

Fsp2_x = Fsp2 & A.x
Fsp2_z = Fsp2 & A.z

X_side = Fsp1_x + Fsp2_x
Z_side = Fsp1_z + Fsp2_z + ((12) * 9.81)

X_init = X_side.subs({x:init_x,
                 z:1.5,
                 H:10.4,
                 a:0.5,
                 b:0.3, 
                 beta:0})

Z_init = Z_side.subs({x:init_x,
                 z:1.5,
                 H:10.4,
                 a:0.5,
                 b:0.3, 
                 beta:0})

Equ_init = [X_init, Z_init]
temp_init= sympy.linsolve(Equ_init,fake_length1,fake_length2)
subtract_L1_init, subtract_L2_init = next(iter(temp_init))

delta_init_L1 = geo_Len1_init - subtract_L1_init
delta_init_L2 = geo_Len2_init - subtract_L2_init

actual_init_L1 = subtract_L1_init - delta_init_L1
actual_init_L2 = subtract_L2_init - delta_init_L2

print('These are the actual lengths\n----------------\
        \nInitial Length 1: {}\nInitial Length 2: {}'.format(actual_init_L1,
                                     actual_init_L2))

These are the actual lengths
----------------        
Initial Length 1: 2.57716402637152
Initial Length 2: 7.62834777870015


In [165]:
Fsp2_z.subs({beta:0, a:0.5,b:0.3,x:5.2, H:10.4, z:1.5})

In [148]:
Z_side.subs({beta:0, a:0.5,b:0.3,x:5.2, H:10.4, z:1.5,
             fake_length1:subtract_L1_init,fake_length2:subtract_L1_init})

In [150]:
delta_init_L1

In [142]:
subtract_L2_init